In [2]:
from __future__ import print_function
import numpy as np
import torch
import torchvision as tv
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader

In [3]:
# defining transforms to resize the images and converting them into a tensor
transform = tv.transforms.Compose([tv.transforms.Resize(227),tv.transforms.ToTensor()])
# dataset used for training and testing
traindata = tv.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testdata = tv.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
batchsize = 128
# load batches of images and the corresponding labels
train_loader = DataLoader(traindata, batch_size=batchsize, shuffle=True)
test_loader = DataLoader(testdata, batch_size=batchsize, shuffle=True)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class CNN(nn.Module):
      def __init__(self):
          super(CNN, self).__init__()
          self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4)
          self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
          self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
          self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
          self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
          self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
          self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
          self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)
          self.drop_out1 = nn.Dropout(0.5)
          self.fc1 = nn.Linear(in_features=9216, out_features=4096)
          self.drop_out2 = nn.Dropout(0.5)
          self.fc2 = nn.Linear(in_features=4096, out_features=4096)
          self.fc3 = nn.Linear(in_features=4096, out_features=10)
      def forward(self, x): 
          output = self.pool1(F.relu(self.conv1(x)))
          output = self.pool2(F.relu(self.conv2(output)))
          output = F.relu(self.conv3(output))
          output = F.relu(self.conv4(output))
          output = self.pool3(F.relu(self.conv5(output)))
          output = output.flatten(1)
          output = self.drop_out1(output)
          output = F.relu(self.fc1(output))
          output = self.drop_out1(output)
          output = F.relu(self.fc2(output))
          output = F.relu(self.fc3(output))
          return output
neu_net = CNN()
if torch.cuda.is_available():
   neu_net.cuda()        

In [5]:
# loss function used and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(neu_net.parameters(), lr=0.01, weight_decay=0.0005, momentum=0.9)

In [18]:
# train the model
for epoch in range(1,11):
    # create a loop to enumerate over batches 
    loss = 0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        # get the input images and labels
        if torch.cuda.is_available():
           inputs, labels = inputs.cuda(), labels.cuda()   
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward propagation 
        outputs = neu_net(inputs)
        # calculate loss 
        loss_func = loss_function(outputs, labels)
        loss += loss_func.item()
        prediction = outputs.argmax(1)  # calculate the index that has maximum value 
        correct_predictions += prediction.eq(labels.data).sum().item()
        total_predictions += labels.size(0)
        # use the backward call to calculate gradients of loss function wrt the parameters
        loss_func.backward()
        # update the values of parameters
        optimizer.step()   
    print('Epoch: %d  Loss: %.5f  Training Accuracy: %.2f %%' % (epoch, loss / (len(train_loader)), 100*(correct_predictions/total_predictions)))
     

Epoch: 1  Loss: 0.31195  Training Accuracy: 87.87 %
Epoch: 2  Loss: 0.30775  Training Accuracy: 87.89 %
Epoch: 3  Loss: 0.30282  Training Accuracy: 88.09 %
Epoch: 4  Loss: 0.30329  Training Accuracy: 88.09 %
Epoch: 5  Loss: 0.30302  Training Accuracy: 88.02 %
Epoch: 6  Loss: 0.30127  Training Accuracy: 88.20 %
Epoch: 7  Loss: 0.29213  Training Accuracy: 88.40 %
Epoch: 8  Loss: 0.29382  Training Accuracy: 88.29 %
Epoch: 9  Loss: 0.28811  Training Accuracy: 88.41 %
Epoch: 10  Loss: 0.28791  Training Accuracy: 88.48 %


In [19]:
# calculate accuracy on test data
correct_predictions = 0
total_predictions = 0
for inputs, labels in test_loader:
    if torch.cuda.is_available():
           inputs, labels = inputs.cuda(), labels.cuda() 
           outputs = neu_net(inputs)
           prediction = outputs.argmax(1)
           correct_predictions += prediction.eq(labels.data).sum().item()
           total_predictions += labels.size(0)
print('Test_Accuracy: %.2f %%' % (100 * (correct_predictions/total_predictions)))

Test_Accuracy: 76.51 %
